# noaa-weather-hourly
This script cleans and formats a manually downloaded "National Oceanic and Atmospheric Administration" (NOAA) "LCD CSV" weather file.

 __Originated From:__
https://github.com/emskiphoto/Process-historical-NOAA-LCD-weather<BR>
    
Copyright December 31, 2024
https://github.com/emskiphoto/noaa-weather-hourly
Matt Chmielewski

### Load Python packages

In [1]:
# direct the jupyter notebook development script to add package path to PYTHONPATH
# to allow for loading of source modules.
import sys
sys.path.append('../noaa-weather-hourly')

In [2]:
from config import *
from utils import *

In [3]:
import pandas as pd
# import matplotlib.pyplot as plt
# import datetime
import pathlib
import re
# import holidays
# from pathlib import PureWindowsPath as PureWindowsPath
# turn off Jedi autocomplete (that was causing more problems than benefits post Win10 update 3-13-2020)
%config Completer.use_jedi = False 

### Parameters - Store in config.py

In [4]:
# freqstr = 'H'
# pattern_lcd_input_file = r'^[0-9]{5,10}.csv'
# pattern_isd_history_file = r'isd-history.csv|ISD-HISTORY.CSV'
# file_output_format = """{STATION_NAME} {from_str} to {end_str} {freqstr}.csv"""
# pct_null_timestamp_max = 0.5

## Locations

In [5]:
dir_cwd = pathlib.Path.cwd()
dir_cwd

WindowsPath('C:/Users/user/OneDrive/python_envs/noaa-weather-hourly-cli/dev')

In [6]:
dir_data = dir_cwd / 'data'
assert dir_data.is_dir()

### Locate LCD .CSV file 'file_lcd_input'
This script is intended to be executed from a terminal command line.  The LCD input file is expected to be saved in the same directory as the command line prompt.  The file name is expected to match the pattern in 'pattern_lcd_input_file'.  However, if a file with this pattern is not identifed, attempt to use the most recent .CSV file in the same directory.

In [7]:
# which version of LCD files are avaialble and which are the most recent?
# 1. find all files that match v1 or v2 naming
# 2. find the most recent file
# 3. Determine if most recent file is v1 or v2 format 'lcd_version'
# 4. see if there is more than one file with the same station ID
# 5. create list 'files_lcd' with one or more lcd files of same station id 

In [49]:
# 1. find all files that match v1 or v2 naming and sort by last modified date descending
version_files = {v_ : find_files_re_pattern_sorted_last_modified(dir_data, pattern_) for
                 v_, pattern_ in version_pattern_lcd_input.items()}
version_files

{1: [WindowsPath('C:/Users/user/OneDrive/python_envs/noaa-weather-hourly-cli/dev/data/3876540.csv'),
  WindowsPath('C:/Users/user/OneDrive/python_envs/noaa-weather-hourly-cli/dev/data/3875753.csv')],
 2: [WindowsPath('C:/Users/user/OneDrive/python_envs/noaa-weather-hourly-cli/dev/data/LCD_USW00014939_2023.csv'),
  WindowsPath('C:/Users/user/OneDrive/python_envs/noaa-weather-hourly-cli/dev/data/LCD_USW00014939_2022.csv'),
  WindowsPath('C:/Users/user/OneDrive/python_envs/noaa-weather-hourly-cli/dev/data/LCD_USW00014939_2021.csv'),
  WindowsPath('C:/Users/user/OneDrive/python_envs/noaa-weather-hourly-cli/dev/data/LCD_USW00014939_2020.csv'),
  WindowsPath('C:/Users/user/OneDrive/python_envs/noaa-weather-hourly-cli/dev/data/LCD_USW00094846_2023.csv'),
  WindowsPath('C:/Users/user/OneDrive/python_envs/noaa-weather-hourly-cli/dev/data/LCD_USW00094846_2022.csv'),
  WindowsPath('C:/Users/user/OneDrive/python_envs/noaa-weather-hourly-cli/dev/data/LCD_USW00094846_2021.csv'),
  WindowsPath('C:/Us

In [64]:
# find most recently modified file by each lcd version
version_file_last_modified = {version_ : files_[0] for version_, files_ in version_files.items()}
version_file_last_modified

{1: WindowsPath('C:/Users/user/OneDrive/python_envs/noaa-weather-hourly-cli/dev/data/3876540.csv'),
 2: WindowsPath('C:/Users/user/OneDrive/python_envs/noaa-weather-hourly-cli/dev/data/LCD_USW00014939_2023.csv')}

In [70]:
# 2. find the most recent file
file_last_modified = sorted([(f, f.stat().st_mtime) for
                  f in version_file_last_modified.values()],
           key=lambda x: x[1], reverse=True)[0][0]
# 3. Determine if most recent file is v1 or v2 format 'lcd_version'
# versions start with '1' so need to add 1 to zero-indexed list
lcd_version = list(version_file_last_modified.values())\
                            .index(file_last_modified) + 1
# file_last_modified, lcd_version

In [71]:
# make sure we have the right version
assert file_last_modified in version_files[lcd_version]

### Load LCD .CSV input file(s)

# STOPPED HERE 12-31-24

In [ ]:
file

In [73]:
#     open csv and store in pandas dataframe
df = pd.read_csv(file_lcd_input, parse_dates=['DATE'], index_col='DATE', low_memory=False)
df

,STATION,REPORT_TYPE,SOURCE,AWND,BackupDirection,BackupDistance,BackupDistanceUnit,BackupElements,BackupElevation,BackupElevationUnit,...,ShortDurationPrecipitationValue045,ShortDurationPrecipitationValue060,ShortDurationPrecipitationValue080,ShortDurationPrecipitationValue100,ShortDurationPrecipitationValue120,ShortDurationPrecipitationValue150,ShortDurationPrecipitationValue180,Sunrise,Sunset,WindEquipmentChangeDate
DATE,,,,,,,,,,,,,,,,,,,,,
2020-01-01 00:00:00,72530094846,FM-12,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020-01-01 00:51:00,72530094846,FM-15,7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020-01-01 01:51:00,72530094846,FM-15,7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020-01-01 02:51:00,72530094846,FM-15,7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020-01-01 03:51:00,72530094846,FM-15,7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-12-31 20:51:00,72530094846,FM-15,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2023-12-31 21:00:00,72530094846,FM-12,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2023-12-31 21:51:00,72530094846,FM-15,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [74]:
cols_use = ['STATION', 
       'HourlyVisibility', 'HourlyDryBulbTemperature', 'HourlyWindSpeed',
       'HourlyDewPointTemperature', 'HourlyRelativeHumidity',
       'HourlyWindDirection', 'HourlyStationPressure',
       'HourlyWetBulbTemperature',
       'HourlyAltimeterSetting',
       'HourlyPrecipitation', 'HourlyPressureChange',
        'HourlyWindGustSpeed', 'Sunset',
       'Sunrise']
cols_use = df.columns.intersection(cols_use).tolist()
cols_not_used = df.columns.difference(cols_use).tolist()
cols_use

['STATION',
 'HourlyAltimeterSetting',
 'HourlyDewPointTemperature',
 'HourlyDryBulbTemperature',
 'HourlyPrecipitation',
 'HourlyPressureChange',
 'HourlyRelativeHumidity',
 'HourlyStationPressure',
 'HourlyVisibility',
 'HourlyWetBulbTemperature',
 'HourlyWindDirection',
 'HourlyWindGustSpeed',
 'HourlyWindSpeed',
 'Sunrise',
 'Sunset']

In [75]:
df = df[cols_use].copy()
df.shape

(51668, 15)

In [76]:
df.head()

,STATION,HourlyAltimeterSetting,HourlyDewPointTemperature,HourlyDryBulbTemperature,HourlyPrecipitation,HourlyPressureChange,HourlyRelativeHumidity,HourlyStationPressure,HourlyVisibility,HourlyWetBulbTemperature,HourlyWindDirection,HourlyWindGustSpeed,HourlyWindSpeed,Sunrise,Sunset
DATE,,,,,,,,,,,,,,,
2020-01-01 00:00:00,72530094846,NaN,21,25,NaN,NaN,85.0,29.10,9.94,24.0,270,NaN,8.0,NaN,NaN
2020-01-01 00:51:00,72530094846,29.80,20,24,0.00,NaN,84.0,29.08,10.00,23.0,270,NaN,8.0,NaN,NaN
2020-01-01 01:51:00,72530094846,29.80,19,23,0.00,NaN,85.0,29.08,10.00,22.0,260,NaN,6.0,NaN,NaN
2020-01-01 02:51:00,72530094846,29.81,19,22,0.00,-0.01,89.0,29.09,10.00,21.0,220,NaN,5.0,NaN,NaN
2020-01-01 03:51:00,72530094846,29.80,19,21,0.00,NaN,92.0,29.08,10.00,20.0,230,NaN,6.0,NaN,NaN


### Identify and Display Weather Station Information

In [77]:
# identify station
station_lcd = str(df['STATION'].iloc[-1])
station_usaf, station_wban = station_lcd[:6], station_lcd[6:]
station_lcd, station_usaf, station_wban

('72530094846', '725300', '94846')

#### Open 'isd-history.csv' containing Station details
This is supplemental, non-essential information.   It is important that any errors with this step do not negatively impact the core process.

In [78]:
isd_history_available = False
file_isd_history = find_latest_file(dir_data, pattern_isd_history_file)
if file_isd_history.is_file():
    isd_history_available = True

In [79]:
df_isd_history = pd.read_csv(file_isd_history, index_col='WBAN',
                             dtype={'WBAN': object}).sort_values(by=['USAF', 'BEGIN'],
                                                             ascending=[True, False])
df_isd_history.sample(5).sort_index()

,USAF,STATION NAME,CTRY,ST,CALL,LAT,LON,ELEV(M),BEGIN,END
WBAN,,,,,,,,,,
00003,049999,GISP2,GL,NaN,NaN,+72.580,-038.460,+3205.0,1989-06-08,1997-01-25
04114,722142,CHALLIS AIRPORT,US,ID,KLLJ,+44.523,-114.216,+1534.1,2005-01-01,2024-12-15
13825,723306,COLUMBUS AFB AIRPORT,US,MS,KCBM,+33.650,-088.450,+0066.8,1942-03-01,2024-12-16
23293,724945,N Y. MINETA SN JO INTL APT,US,CA,KSJC,+37.359,-121.924,+0015.0,1973-01-01,2024-12-15
53952,722563,MC GREGOR EXECUTIVE ARPT,US,TX,KPWG,+31.485,-097.316,+0180.4,2006-01-01,2024-12-15


In [80]:
station_wban

'94846'

In [81]:
# df_isd_history.loc[int(station_wban)]
# using station_wban, gather station details from isd history for the record with the most
# recent 'END' date
station_details = dict(df_isd_history.loc[station_wban]\
                       .sort_values('END', ascending=False).iloc[0])

In [82]:
# df_isd_history.groupby('WBAN')['USAF'].count().value_counts()

# df_isd_history.groupby('WBAN')['USAF'].count().sort_values(ascending=False)

# df_isd_history.loc['03849']

In [83]:
# remove 'STATION', 'REPORT_TYPE', 'SOURCE' columns - not needed anymore
df.drop(columns=['STATION', 'REPORT_TYPE', 'SOURCE'], inplace=True, errors='ignore')

In [84]:
#     create timestamps for weather data time series
start_dt = df.index[0]
end_dt = df.index[-1]
start_str = start_dt.strftime('%Y-%m-%d')
end_str = end_dt.strftime('%Y-%m-%d')
start_str, end_str

('2020-01-01', '2023-12-31')

In [85]:
df.dtypes

HourlyAltimeterSetting        object
HourlyDewPointTemperature     object
HourlyDryBulbTemperature      object
HourlyPrecipitation           object
HourlyPressureChange         float64
HourlyRelativeHumidity       float64
HourlyStationPressure         object
HourlyVisibility              object
HourlyWetBulbTemperature     float64
HourlyWindDirection           object
HourlyWindGustSpeed          float64
HourlyWindSpeed              float64
Sunrise                      float64
Sunset                       float64
dtype: object

In [86]:
cols_dtypes = {'HourlyAltimeterSetting': 'float64',
   'HourlyDewPointTemperature': 'float64',
    'HourlyDryBulbTemperature': 'float64',
    'HourlyPrecipitation': 'float64',
    'HourlyPressureChange': 'float64',
    'HourlyRelativeHumidity': 'Int64',
    'HourlyStationPressure': 'float64',
    'HourlyVisibility': 'float64',
    'HourlyWetBulbTemperature': 'float64',
    'HourlyWindDirection': 'Int64',
    'HourlyWindGustSpeed': 'float64',
    'HourlyWindSpeed': 'float64',
    'Sunrise': 'datetime64[ns]',
    'Sunset': 'datetime64[ns]'}
# keep only items with corresponding column in df
cols_dtypes = {k : v for k, v in cols_dtypes.items() if k in df.columns}

In [87]:
for col_, type_ in cols_dtypes.items():
    if type_ == 'float64':
        df[col_] = pd.to_numeric(df[col_], errors='coerce')
        try:
            df[col_] = df[col_].astype(type_)
        except:
            pass
    elif type_ == 'Int64':
        df[col_] = pd.to_numeric(df[col_], errors='coerce')
        try:
            df[col_] = df[col_].astype(type_)
        except:
            pass

In [88]:
df.dtypes

HourlyAltimeterSetting       float64
HourlyDewPointTemperature    float64
HourlyDryBulbTemperature     float64
HourlyPrecipitation          float64
HourlyPressureChange         float64
HourlyRelativeHumidity         Int64
HourlyStationPressure        float64
HourlyVisibility             float64
HourlyWetBulbTemperature     float64
HourlyWindDirection            Int64
HourlyWindGustSpeed          float64
HourlyWindSpeed              float64
Sunrise                      float64
Sunset                       float64
dtype: object

In [89]:
# forward fill 'Sunrise' & 'Sunset' data
cols_sunrise_sunset = df.columns.intersection(['Sunrise', 'Sunset']).tolist()
for col_ in cols_sunrise_sunset:
    print(col_)
    time = pd.to_datetime(pd.to_numeric(df[col_].ffill()\
                             .bfill()), format = '%H%M')
    YMDHM = pd.DataFrame({'Year': df.index.year,
                         'Month' : df.index.month,
                        'Day' : df.index.day,
                        'Hour' : time.dt.hour,
                         'Minute' : time.dt.minute})

    df[col_] = pd.to_datetime(YMDHM)
    del time, YMDHM
df[cols_sunrise_sunset].sample(5)

Sunrise
Sunset


,Sunrise,Sunset
DATE,,
2021-01-23 22:51:00,2021-01-23 07:13:00,2021-01-23 16:54:00
2023-03-16 06:51:00,2023-03-16 06:04:00,2023-03-16 17:58:00
2023-08-20 00:00:00,2023-08-20 05:04:00,2023-08-20 18:46:00
2023-03-23 20:51:00,2023-03-23 05:52:00,2023-03-23 18:06:00
2022-10-21 20:51:00,2022-10-21 06:10:00,2022-10-21 17:03:00


In [90]:
# are there timestamps that have a high count of null values?
n_records_hourly_approx = int(df.shape[0]/(24))
n_max_null = pct_null_timestamp_max * n_records_hourly_approx

In [91]:
temp = df.loc[:, df.columns.difference(cols_sunrise_sunset)]
df_nan_ts = temp.groupby(temp.index.time).apply(lambda x: x.isna().sum()\
                            .gt(n_max_null)).all(axis=1)
times_nan = df_nan_ts.loc[df_nan_ts].index.tolist()
del temp
del df_nan_ts

In [92]:
times_nan

[datetime.time(23, 59)]

In [93]:
# remove records for timestamps with a high percentage of Null values.
# note that the '23:59:00' timestamp is suspect and appears to only be a placeholder
# for posting sunrise/sunset times.  Important that this step be done after
# forward filling sunrise/sunset values.
filter_nan_times = pd.Series(df.index.time).isin(times_nan).values
df = df.loc[~filter_nan_times]
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 50154 entries, 2020-01-01 00:00:00 to 2023-12-31 23:51:00
Data columns (total 14 columns):
 #   Column                     Non-Null Count  Dtype         
---  ------                     --------------  -----         
 0   HourlyAltimeterSetting     44163 non-null  float64       
 1   HourlyDewPointTemperature  50138 non-null  float64       
 2   HourlyDryBulbTemperature   50146 non-null  float64       
 3   HourlyPrecipitation        33893 non-null  float64       
 4   HourlyPressureChange       17501 non-null  float64       
 5   HourlyRelativeHumidity     50139 non-null  Int64         
 6   HourlyStationPressure      49991 non-null  float64       
 7   HourlyVisibility           50106 non-null  float64       
 8   HourlyWetBulbTemperature   49993 non-null  float64       
 9   HourlyWindDirection        49554 non-null  Int64         
 10  HourlyWindGustSpeed        9990 non-null   float64       
 11  HourlyWindSpeed            50150

In [94]:
#     Check what percentage of data has null data and print to screen
df_pct_null_data = pd.DataFrame({'Percent N/A': df.isnull().sum().divide(len(df)).round(3)})
df_pct_null_data_formatted = df_pct_null_data['Percent N/A'].apply(lambda n: '{:,.1%}'.format(n))
# remove 'Hourly' prefix for display only
col_rename_remove_hourly = {col_ : col_.replace('Hourly', '') for
                            col_ in df_pct_null_data_formatted.index}

In [95]:
message_pct_null_data = f"""Percent Missing Values by Column for LCD source file '{file_lcd_input.name}' for USAF station {station_usaf} at '{station_details['STATION NAME']}' from {start_str} to {end_str}."""
print(message_pct_null_data)
display(df_pct_null_data_formatted.rename(index=col_rename_remove_hourly))

Percent Missing Values by Column for LCD source file '3876540.csv' for USAF station 725300 at 'CHICAGO O\'HARE INTERNATIONAL' from 2020-01-01 to 2023-12-31.


AltimeterSetting       11.9%
DewPointTemperature     0.0%
DryBulbTemperature      0.0%
Precipitation          32.4%
PressureChange         65.1%
RelativeHumidity        0.0%
StationPressure         0.3%
Visibility              0.1%
WetBulbTemperature      0.3%
WindDirection           1.2%
WindGustSpeed          80.1%
WindSpeed               0.0%
Sunrise                 0.0%
Sunset                  0.0%
Name: Percent N/A, dtype: object

In [96]:
# create dictionary 'col_null_policy' to manually guide the resolution of null values by column
# in general, data that is expected to have a non-zero/non-null reading at all times
# will have nulls filled through interpolation.  Data that normally has 0 or null readings
# like 'HourlyPrecipitation' will have null values filled with 0 in the interest of having
# a 100% clean dataset to facilitate manual analysis.
col_null_policy = {'HourlyAltimeterSetting': pd.Series.interpolate,
 'HourlyDewPointTemperature': pd.Series.interpolate,
 'HourlyDryBulbTemperature': pd.Series.interpolate,
 'HourlyPrecipitation': lambda x: x.fillna(0),
 'HourlyPressureChange': pd.Series.interpolate,
 'HourlyRelativeHumidity': pd.Series.interpolate,
 'HourlyStationPressure': pd.Series.interpolate,
 'HourlyVisibility': pd.Series.interpolate,
 'HourlyWetBulbTemperature': pd.Series.interpolate,
 'HourlyWindDirection': pd.Series.interpolate,
 'HourlyWindGustSpeed': lambda x: x,
 'HourlyWindSpeed': pd.Series.interpolate,
 'Sunrise': pd.Series.ffill,
 'Sunset': pd.Series.bfill}
# keep only items with corresponding column in df
col_null_policy = {k : v for k, v in col_null_policy.items() if k in df.columns}
# assert len(df.columns.intersection(col_null_policy.keys())) == df.shape[1]

In [97]:
df.dtypes

HourlyAltimeterSetting              float64
HourlyDewPointTemperature           float64
HourlyDryBulbTemperature            float64
HourlyPrecipitation                 float64
HourlyPressureChange                float64
HourlyRelativeHumidity                Int64
HourlyStationPressure               float64
HourlyVisibility                    float64
HourlyWetBulbTemperature            float64
HourlyWindDirection                   Int64
HourlyWindGustSpeed                 float64
HourlyWindSpeed                     float64
Sunrise                      datetime64[ns]
Sunset                       datetime64[ns]
dtype: object

In [98]:
# df['HourlyWindSpeed'].astype(float).interpolate()

In [99]:
df['HourlyAltimeterSetting']

DATE
2020-01-01 00:00:00      NaN
2020-01-01 00:51:00    29.80
2020-01-01 01:51:00    29.80
2020-01-01 02:51:00    29.81
2020-01-01 03:51:00    29.80
                       ...  
2023-12-31 20:51:00    30.08
2023-12-31 21:00:00      NaN
2023-12-31 21:51:00    30.09
2023-12-31 22:51:00    30.11
2023-12-31 23:51:00    30.13
Name: HourlyAltimeterSetting, Length: 50154, dtype: float64

In [100]:
col = 'HourlyAltimeterSetting'
# df[col].resample('H').agg(col_null_policy[col])

pd.to_numeric(df[col], errors='coerce').dropna().resample(freqstr).mean()

DATE
2020-01-01 00:00:00    29.800
2020-01-01 01:00:00    29.800
2020-01-01 02:00:00    29.810
2020-01-01 03:00:00    29.800
2020-01-01 04:00:00    29.800
                        ...  
2023-12-31 19:00:00    30.060
2023-12-31 20:00:00    30.075
2023-12-31 21:00:00    30.090
2023-12-31 22:00:00    30.110
2023-12-31 23:00:00    30.130
Freq: H, Name: HourlyAltimeterSetting, Length: 35064, dtype: float64

In [109]:
# create dictionary 'col_resample_policy' to manually guide resampling method by column

col_resample_policy = {'HourlyAltimeterSetting': 'mean',
 'HourlyDewPointTemperature': 'mean',
 'HourlyDryBulbTemperature': 'mean',
 'HourlyPrecipitation': 'mean',
 'HourlyPressureChange': 'mean',
 'HourlyRelativeHumidity': 'mean',
 'HourlyStationPressure': 'mean',
 'HourlyVisibility': 'mean',
 'HourlyWetBulbTemperature': 'mean',
 'HourlyWindDirection': 'mean',
 'HourlyWindGustSpeed': 'mean',
 'HourlyWindSpeed': 'mean',
 'Sunrise': pd.Series.mode,
 'Sunset': pd.Series.mode}
# keep only items with corresponding column in df
col_resample_policy = {k : v for k, v in col_resample_policy.items() if k in df.columns}
# assert len(df.columns.intersection(col_resample_policy.keys())) == df.shape[1]

In [102]:
df['Sunrise'].dtype.name

'datetime64[ns]'

# STOPPED HERE

In [ ]:
pd.Series.resample()

In [110]:
%%time
dfs = {}
for col_ in df.columns:
    print(col_)
#     if df[col_].dtype.name == 'datetime64[ns]':
#         df[col_].resample(freqstr).agg(col_resample_policy[col_])
#     else:
#     df[col_].agg(col_null_policy[col_])
#         pd.to_numeric(df[col], errors='coerce').dropna().resample(freqstr).agg(col_resample_policy[col_])
    dfs[col_] = df[col_].dropna().resample(freqstr).agg(col_resample_policy[col_])

HourlyAltimeterSetting
HourlyDewPointTemperature
HourlyDryBulbTemperature
HourlyPrecipitation
HourlyPressureChange
HourlyRelativeHumidity
HourlyStationPressure
HourlyVisibility
HourlyWetBulbTemperature
HourlyWindDirection
HourlyWindGustSpeed
HourlyWindSpeed
Sunrise
Sunset
CPU times: total: 36.8 s
Wall time: 37 s


In [111]:
df_out = pd.concat(dfs, axis=1)
df_out

,HourlyAltimeterSetting,HourlyDewPointTemperature,HourlyDryBulbTemperature,HourlyPrecipitation,HourlyPressureChange,HourlyRelativeHumidity,HourlyStationPressure,HourlyVisibility,HourlyWetBulbTemperature,HourlyWindDirection,HourlyWindGustSpeed,HourlyWindSpeed,Sunrise,Sunset
DATE,,,,,,,,,,,,,,
2020-01-01 00:00:00,29.800,20.5,24.5,0.0,NaN,84.5,29.09,9.97,23.5,270.0,NaN,8.0,2020-01-01 07:20:00,2020-01-01 16:31:00
2020-01-01 01:00:00,29.800,19.0,23.0,0.0,NaN,85.0,29.08,10.00,22.0,260.0,NaN,6.0,2020-01-01 07:20:00,2020-01-01 16:31:00
2020-01-01 02:00:00,29.810,19.0,22.0,0.0,-0.01,89.0,29.09,10.00,21.0,220.0,NaN,5.0,2020-01-01 07:20:00,2020-01-01 16:31:00
2020-01-01 03:00:00,29.800,19.0,21.0,0.0,NaN,92.0,29.08,10.00,20.0,230.0,NaN,6.0,2020-01-01 07:20:00,2020-01-01 16:31:00
2020-01-01 04:00:00,29.800,19.0,21.0,0.0,NaN,92.0,29.08,10.00,20.0,230.0,NaN,7.0,2020-01-01 07:20:00,2020-01-01 16:31:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-12-31 19:00:00,30.060,27.0,33.0,NaN,NaN,78.0,NaN,10.00,NaN,320.0,NaN,16.0,2023-12-31 07:19:00,2023-12-31 16:29:00
2023-12-31 20:00:00,30.075,27.5,33.0,NaN,-0.06,80.0,NaN,10.00,NaN,320.0,24.0,16.0,2023-12-31 07:19:00,2023-12-31 16:29:00
2023-12-31 21:00:00,30.090,27.0,33.0,NaN,-0.06,78.0,29.37,9.97,31.0,325.0,22.0,14.5,2023-12-31 07:19:00,2023-12-31 16:29:00


In [112]:
df_out.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 35064 entries, 2020-01-01 00:00:00 to 2023-12-31 23:00:00
Freq: H
Data columns (total 14 columns):
 #   Column                     Non-Null Count  Dtype         
---  ------                     --------------  -----         
 0   HourlyAltimeterSetting     35062 non-null  float64       
 1   HourlyDewPointTemperature  35058 non-null  float64       
 2   HourlyDryBulbTemperature   35063 non-null  float64       
 3   HourlyPrecipitation        31779 non-null  float64       
 4   HourlyPressureChange       17501 non-null  float64       
 5   HourlyRelativeHumidity     35058 non-null  Float64       
 6   HourlyStationPressure      34957 non-null  float64       
 7   HourlyVisibility           35063 non-null  float64       
 8   HourlyWetBulbTemperature   34952 non-null  float64       
 9   HourlyWindDirection        34698 non-null  Float64       
 10  HourlyWindGustSpeed        7955 non-null   float64       
 11  HourlyWindSpeed         

In [121]:
# df_out[cols_sunrise_sunset].diff(axis=1).iloc[:,-1].dt.total_seconds().div(3600).plot()

In [ ]:
# df.agg(col_null_policy).astype(cols_dtypes)
# df.agg(col_null_policy).isna().sum()

In [ ]:
# transform remaining null values per the 'col_null_policy'
# df = df.agg(col_null_policy).astype(cols_dtypes)
# display(pd.DataFrame({'Percent N/A': df.isnull().sum().divide(len(df))\
#                       .round(3)}).apply(lambda n: '{:,.1%}'.format(n)))

In [122]:
#     Check what percentage of data has null data and print to screen
df_pct_null_data_post = pd.DataFrame({'Percent N/A': df_out.isnull().sum().divide(len(df_out)).round(3)})
df_pct_null_data_post_formatted = df_pct_null_data_post['Percent N/A'].apply(lambda n: '{:,.1%}'.format(n))
display(df_pct_null_data_post_formatted)

HourlyAltimeterSetting        0.0%
HourlyDewPointTemperature     0.0%
HourlyDryBulbTemperature      0.0%
HourlyPrecipitation           9.4%
HourlyPressureChange         50.1%
HourlyRelativeHumidity        0.0%
HourlyStationPressure         0.3%
HourlyVisibility              0.0%
HourlyWetBulbTemperature      0.3%
HourlyWindDirection           1.0%
HourlyWindGustSpeed          77.3%
HourlyWindSpeed               0.0%
Sunrise                       0.0%
Sunset                        0.0%
Name: Percent N/A, dtype: object

In [123]:
df_pct_null_data_formatted

HourlyAltimeterSetting       11.9%
HourlyDewPointTemperature     0.0%
HourlyDryBulbTemperature      0.0%
HourlyPrecipitation          32.4%
HourlyPressureChange         65.1%
HourlyRelativeHumidity        0.0%
HourlyStationPressure         0.3%
HourlyVisibility              0.1%
HourlyWetBulbTemperature      0.3%
HourlyWindDirection           1.2%
HourlyWindGustSpeed          80.1%
HourlyWindSpeed               0.0%
Sunrise                       0.0%
Sunset                        0.0%
Name: Percent N/A, dtype: object

In [ ]:
for col_ in df.columns:
    print(col_)
    df[col_].resample(freqstr)

In [ ]:
df.dtypes

In [ ]:
pd.to_numeric(df['HourlyVisibility'])

In [ ]:
pd.to_datetime(df['Sunrise'])

In [ ]:
df.resample(freqstr).mean()

In [ ]:
#     Resample data to produce evenly spaced interval data
# df = df.resample(freqstr).median()
df = df.resample(freqstr).mean()

### Create Additional Columns

In [ ]:
# #     assign seasons to df in new 'season' column
#     df = weather_assign_season(df)
# #     assign weekend or weekday tag to each day
#     df['weekday'] = 'weekday'
#     df.loc[~df.index.weekday.isin(list(range(0,5))),'weekday'] = 'weekend'
#     df['weekday'] = df['weekday'].astype('category')   # keep memory size to a minimum
# #     Add True/False Peak Coincident column
#     df = is_month_and_hour_in_list(df, new_column='Peak Coincident', months=[6,7,8,9],hours=[14,15,16,17])
# #     Add holiday True/False:
#     years = list(df.index.year.unique())
#     try:
#         holiday_dates = generate_list_of_holidays(years=years)
#         holiday_YMD = []
#         for d in holiday_dates:
#             holiday_YMD.append(datetime.datetime(d.year, d.month, d.day).date())
#         df['holiday'] = [dat in holiday_YMD for dat in df.index.date]
# #         alternatively:
# # holiday_dates = generate_list_of_holidays(years=df.index.year.unique().to_list())
# # df['Date_copy_delete'] = df.index.date.copy()
# # df['holiday'] = df.Date_copy_delete.apply(lambda date: date in holiday_dates)
# # df.drop(columns=['Date_copy_delete'], inplace=True)
#     except:
#         print("Holidays were not tagged.  Veritfy it 'Holidays' python package is installed")
# #     Add temperature bins
#     temp_bins = list(range(-30,110,5))
#     df['DryBulbTemperature_bin'] = pd.cut(df['HourlyDryBulbTemperature'],bins=temp_bins)

### Export df to Disk

#### Rename Columns - remove 'hourly' from names

#### Name export file 

In [ ]:
# city = input("Input city or project name:  ")
# export_name = "{}_".format(city) + start_str + "_to_" + end_str + f"_{df.index.freqstr}.csv"
# export_name

In [7]:
file_output_format.format(STATION_NAME = 'billy',
                         )

KeyError: 'from_str'

In [ ]:
file_out_name = f"{station_details['']city) + start_str + "_to_" + end_str + f"_{df.index.freqstr}.csv"
file_out = dir_cwd / file_out_name
file_out_name

In [ ]:
# os.path.join(os.path.dirname(file),export_name)

In [ ]:
[col.replace('Hourly','') for col in df.columns]

In [ ]:
df.columns

In [ ]:
df.to_csv(os.path.join(os.path.dirname(file),export_name))
# os.path.join(os.path.dirname(directory),f'All_M_V_data {todays_date}.csv')

# END